In [1]:
#TxtFile to lists
import numpy as np 
import matplotlib.pyplot as plt

#data = np.loadtxt('C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\data.txt')

values_list = []  # create an empty list to store the values for each row
x = []
z = []
m = []
l = []

with open("data.txt", "r") as f:
    for line in f:  # loop over each line in the file
        values = line.strip().split("\t")  # split the line into a list of strings

        xL = []
        zL = []
        mL = []
        lL = []

        for i in range(20):
            xL.append(round(float(values[i]),3))
            zL.append(round(float(values[i+20]),3))
            mL.append(round(float(values[i+20*2]),3))
            try:
                lL.append(round(float(values[i+20*3]),3))
            except:
                None
        x.append(xL)
        z.append(zL)
        m.append(mL)
        l.append(lL)


dataY = []

dataX = []
for i in range(len(x)):
    dataXs = []
    for j in range(0,len(x[i])):
        dataX0 = []
        dataX0.append(j)
        dataX0.append(x[i][j])
        dataX0.append(z[i][j])
        dataXs.append(dataX0)
    dataX.append(dataXs)

dataX = np.array(dataX)
dataY = np.array(m)

dataEgdeIndex = []
for i in range(len(x[0])-1):
    dataEgdeIndex.append([i,i+1])
    dataEgdeIndex.append([i+1,i])  

dataEgdeIndex = np.array(dataEgdeIndex) 
dataEgdeIndex =np.transpose(dataEgdeIndex)


input_data = dataX
target_data = dataY
edge_index = dataEgdeIndex


In [2]:
print(edge_index)
print(dataX[0])

[[ 0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12
  12 13 13 14 14 15 15 16 16 17 17 18 18 19]
 [ 1  0  2  1  3  2  4  3  5  4  6  5  7  6  8  7  9  8 10  9 11 10 12 11
  13 12 14 13 15 14 16 15 17 16 18 17 19 18]]
[[  0.      0.      0.   ]
 [  1.      1.023  12.345]
 [  2.      4.064  24.352]
 [  3.      9.039  35.696]
 [  4.     15.814  46.066]
 [  5.     24.204  55.179]
 [  6.     33.979  62.787]
 [  7.     44.873  68.683]
 [  8.     56.589  72.705]
 [  9.     68.807  74.744]
 [ 10.     81.193  74.744]
 [ 11.     93.411  72.705]
 [ 12.    105.127  68.683]
 [ 13.    116.021  62.787]
 [ 14.    125.796  55.179]
 [ 15.    134.186  46.066]
 [ 16.    140.961  35.696]
 [ 17.    145.936  24.352]
 [ 18.    148.977  12.345]
 [ 19.    150.      0.   ]]


In [3]:
print(dataX.shape)
print(dataY.shape)
print(edge_index.shape)
print(edge_index)

from torch_geometric.data import Dataset, Data
import torch
dataset = []
for i in range(dataX.shape[0]):
    dataset.append(Data(dataX[i], edge_index, dataY[i]))

print(dataset)

(286, 20, 3)
(286, 20)
(2, 38)
[[ 0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12
  12 13 13 14 14 15 15 16 16 17 17 18 18 19]
 [ 1  0  2  1  3  2  4  3  5  4  6  5  7  6  8  7  9  8 10  9 11 10 12 11
  13 12 14 13 15 14 16 15 17 16 18 17 19 18]]
[Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge_attr=[20]), Data(x=[20, 3], edge_index=[2, 38], edge

In [98]:
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.data import DataLoader
from torch.utils.data import DataLoader
import os



class ArchDataset(torch.utils.data.Dataset):
    def __init__(self, input_data, edge_index, target_data, scenario_ids):
        self.input_data = input_data
        self.edge_index = edge_index
        self.target_data = target_data
        self.scenario_ids = scenario_ids

    def __getitem__(self, index):
        node_features = torch.tensor(self.input_data[self.scenario_ids[index]], dtype=torch.float)
        edge_index = torch.tensor(self.edge_index, dtype=torch.long)
        y = torch.tensor(self.target_data[self.scenario_ids[index]], dtype=torch.float)
        data = Data(x=node_features, edge_index=edge_index, y=y)
        return data

    def __len__(self):
        return len(self.scenario_ids)
    
num_scenarios = len(input_data)
datasets = []
for i in range(num_scenarios):
    dataset = ArchDataset(input_data, edge_index , target_data, [i] * len(input_data))
    datasets.append(dataset)

# Split into train and test sets
train_dataset = torch.utils.data.ConcatDataset(datasets[:num_scenarios//2])
test_dataset = torch.utils.data.ConcatDataset(datasets[num_scenarios//2:])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)




#train_features, train_labels = next(iter(train_loader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")


In [101]:
print(len(train_dataset))


40898


In [77]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GNN(MessagePassing):
    def __init__(self):
        super(GNN, self).__init__(aggr='mean')
        self.lin1 = torch.nn.Linear(3, 16)
        self.lin2 = torch.nn.Linear(16, 1)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = F.relu(x)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        deg = degree(edge_index, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[edge_index[1]] * deg_inv_sqrt[edge_index[0]]
        x = self.propagate(edge_index, x=x, norm=norm)
        x = self.lin2(x)
        return x

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

In [79]:
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch.nn import MSELoss

# Define model
model = GNN()

# Define loss function and optimizer
criterion = MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(len(dataX)):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch %d, loss: %.3f" % (epoch+1, running_loss / len(train_loader)))
    
# Evaluate the model
total_loss = 0.0
for i, data in enumerate(test_loader, 0):
    outputs = model(data)
    loss = criterion(outputs, data.y)
    total_loss += loss.item()
print("Test loss: %.3f" % (total_loss / len(test_loader)))




IndexError: index 69 is out of bounds for axis 0 with size 0